In [2]:
import numpy as np
import jax.numpy as jnp

import jax
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import rho_plus as rp

is_dark = True
theme, cs = rp.mpl_setup(is_dark)
rp.plotly_setup(is_dark)

In [3]:
%cd ~/cdv

/home/nmiklaucic/cdv


In [19]:
clean = pd.read_pickle('precomputed/jarvis_dft3d_cleaned/dataframe.pkl')
clean

,space_group,formula,e_form,bandgap,atoms,e_total,ehull,dimensionality,density,num_atoms,num_spec,magmom
0,129,"(Ti, Cu, Si, As)",-0.42762,0.000,"[[2.67519992 2.67519992 7.37609819] Ti, [0.891...",-3.37474,0.0423,3D-bulk,5.956,8,4,0.0
1,221,"(Dy, B)",-0.41596,0.000,"[[0. 0. 0.] Dy, [0.81214866 2.04453946 2.04453...",-5.79186,0.0456,3D-bulk,5.522,7,2,0.0
2,119,"(Be, Os, Ru)",0.04847,0.000,"[[0. 0. 0.] Be, [1.83359072 0. 1.74248...",-4.65924,0.3183,intercalated ion,10.960,4,3,0.0
3,14,"(K, Bi)",-0.44140,0.472,"[[0.91524308 6.85585362 9.07269063] K, [ 4.624...",-0.42496,0.0000,intercalated ion,5.145,32,2,0.0
4,164,"(V, Se)",-0.71026,0.000,"[[0. 0. 0.] V, [ 1.67774838 -0.96865035 4.652...",-3.87823,0.0156,2D-bulk,5.718,3,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
75980,164,"(Nd, Mg, Sb)",-0.69038,0.000,"[[0. 0. 0.] Nd, [2.3394364 1.35070495 2.69383...",-1.53096,0.0000,3D-bulk,5.116,5,3,0.0
75983,216,"(Yb, Pr, Pd)",-0.56635,0.000,"[[5.25036997 5.25036997 5.25036997] Yb, [1.750...",-1.70796,0.0000,3D-bulk,8.139,3,3,0.0
75984,216,"(Tb, Tl, Zn)",-0.29921,0.000,"[[1.66349368 1.66349368 1.66349368] Tb, [4.990...",0.05135,0.0000,3D-bulk,9.666,3,3,0.0
75989,216,"(Pr, Tl, Zn)",-0.34112,0.000,"[[1.71052992 1.71052992 1.71052992] Pr, [5.131...",-0.10696,0.0000,3D-bulk,8.517,3,3,0.0


In [41]:
from pymatgen.core import Structure
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import CrystalNN
from tqdm import tqdm
import warnings
nn = CrystalNN()

from multiprocessing import Pool

warnings.simplefilter('ignore', category=UserWarning)

with Pool(processes=48) as P:
    graphs = P.map(nn.get_bonded_structure, clean['atoms'])

/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Exception ignored in: <function _releaseLock at 0x7fc1c8f31620>
Traceback (most recent call last):
  File "/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/logging/__init__.py", line 243, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


In [5]:
elements = set()
for struct in clean['atoms']:
    elements.update(set(struct.elements))

print(' '.join([e.symbol for e in sorted(elements)]))
# print(elements)
# print({e.symbol: (e.average_ionic_radius, e.atomic_radius) for e in elements})

K Rb Ba Na Sr Li Ca La Tb Yb Ce Pr Nd Sm Dy Y Ho Er Tm Hf Mg Zr Sc U Ta Ti Mn Be Nb Al Tl V Zn Cr Cd In Ga Fe Co Cu Si Ni Ag Sn Hg Ge Bi B Sb Te Mo As P H Ir Os Pd Ru Pt Rh Pb W Au C Se S I Br N Cl O F


In [20]:
import pickle

if 'graphs' in globals() or 'graphs' in locals():
    print('Saving...')
    with open('precomputed/jarvis_dft3d_cleaned/graphs.pkl', 'wb') as out:
        pickle.dump(graphs, out)
else:
    print('Loading...')
    with open('precomputed/jarvis_dft3d_cleaned/graphs.pkl', 'rb') as out:
        graphs = pickle.load(out)

Saving...


We want to split the samples into batches. Because Jax needs fixed shapes, we're going to pad
batches to the same number of nodes and edges. This means that we want to sort into batches such
that the max number of edges in a batch is minimized, ideally just below a power of 2. That's what
the below code does.

In [29]:
from tqdm import tqdm
import functools as ft


# each edge is split into 2 directed edges
sizes = np.array([(len(g.graph.nodes), 2 * len(g.graph.edges)) for g in graphs[:-3]])

def get_parts(numbers, batch, chunk_size):    
    # assert len(numbers) % (batch * chunk_size) == 0
    n_batches = len(numbers) // batch
    parts = np.zeros((batch, n_batches), dtype=jnp.int32)
    part_sizes = np.array([0 for _ in range(n_batches)])
    
    chunk_i = 0
    for sample_is in tqdm(np.argsort(-numbers).reshape(batch // chunk_size, chunk_size * n_batches)):
        sample_sizes = numbers[sample_is]
        n_filled = np.zeros((n_batches,), dtype=jnp.int32)
        for sample_i, sample_size in zip(sample_is, sample_sizes):
            next_i = np.argmin(part_sizes + 10000 * (n_filled == chunk_size))
            parts[chunk_i * chunk_size + n_filled[next_i], next_i] += sample_i
            n_filled[next_i] += 1
            part_sizes[next_i] += sample_size
        chunk_i += 1

    return parts, part_sizes

parts, part_sizes = get_parts(sizes[:, 1], 63, 63)
print(part_sizes)

100%|██████████| 1/1 [00:00<00:00,  1.32it/s]

[3732 3732 3732 ... 3728 3728 3728]


In [17]:
70434 / 63

1118.0

In [30]:
# max node size
max(jnp.take(sizes[:, 1], parts).sum(axis=0))

Array(3732, dtype=int32)

In [31]:
jnp.save('precomputed/jarvis_dft3d_cleaned/batches.npy', parts)

In [34]:
recv = jnp.array([4, 1, 1, 2, 2, 3])
edge = jnp.array([0, 2, 3, 1, 1, 4])
nodes = jnp.array([4, 3, 2, 1, 0])

In [38]:
jax.ops.segment_sum(recv, edge)

Array([4, 4, 1, 1, 3], dtype=int32)

In [47]:
'{:.4g}'.format(3.14159)

'3.142'

In [53]:
np.array([0]).dtype.kind

'i'